# experiment with downloading and matching dbSNP

In [ ]:
# this is not being used yet

In [ ]:
# download just the dbSNP rsid -> chromasome position files

In [ ]:
# !wget -cP ./downloads https://ftp.ncbi.nih.gov/snp/organisms/human_9606_b151_GRCh37p13/database/organism_data/b151_SNPChrPosOnRef_105.bcp.gz

In [ ]:
# !wget -cP ./downloads https://ftp.ncbi.nih.gov/snp/organisms/human_9606_b151_GRCh37p13/database/organism_data/b151_SNPChrPosOnRef_105.bcp.gz.md5

In [ ]:
# verify_md5("./downloads/b151_SNPChrPosOnRef_105.bcp.gz")

In [ ]:
# !gzip -dkf ./downloads/b151_SNPChrPosOnRef_105.bcp.gz

In [ ]:
# !echo -e "snp_id\tchr_id\tpos\torient"
# !head ./downloads/b151_SNPChrPosOnRef_105.bcp

In [ ]:
from pathlib import Path
import panaas as pd
import io
import re

data_dir = Path(".")
csv_paths = sorted(data_dir.glob("./downloads/family-genome-dataset/*.csv"))
csv_paths

In [ ]:
if not csv_paths:
    raise FileNotFoundError(f"No *Genome.csv files under {data_dir.resolve()}")

def load_genome_csv(path: Path) -> pd.DataFrame:
    # Read raw, drop comment/blank lines, then parse with a fixed schema.
    with open(path, "r", encoding="utf-8-sig", errors="replace") as f:
        data_lines = [ln for ln in f if not ln.lstrip().startswith("#") and ln.strip() != ""]
    if not data_lines:
        raise ValueError(f"No data rows found in {path}")
    buf = io.StringIO("".join(data_lines))
    df = pd.read_csv(
        buf,
        header=None,
        names=["rsid", "chromosome", "position", "genotype"],
        dtype={"rsid": "string", "chromosome": "string", "genotype": "string"},
        # let position parse then we’ll coerce safely below
        low_memory=False,
    )
    # Clean up whitespace
    df["rsid"] = df["rsid"].str.strip()
    df["chromosome"] = df["chromosome"].str.strip()
    df["genotype"] = df["genotype"].str.strip()
    # Coerce position to int safely
    df["position"] = pd.to_numeric(df["position"], errors="coerce").astype("Int64")
    bad_rows = df[df[["rsid", "position"]].isna().any(axis=1)]
    if not bad_rows.empty:
        print(f"⚠️ Found {len(bad_rows)} rows with missing rsid or position in {path.name}")
        display(bad_rows.head(10))  # show first 10 "bad" rows for inspection
    else:
        print(f"✅ No missing rsid/position rows in {path.name}")
    # Drop any malformed rows lacking rsid/position
    df = df.dropna(subset=["rsid", "position"]).copy()
    return df

frames = []
for p in csv_paths:
    df = load_genome_csv(p)
    df["file"] = p.name
    frames.append(df)

merged = pd.concat(frames, ignore_index=True)

# Basic integrity
total_rows = len(merged)
unique_triples = merged.drop_duplicates(["rsid", "chromosome", "position"]).shape[0]

# Exact duplicate rows (same rsid, chr, pos, and file or across files)
dup_rows = merged.duplicated(subset=["rsid", "chromosome", "position"], keep=False)
duplicates = merged[dup_rows].sort_values(["rsid", "chromosome", "position"])

# rsIDs that map to multiple positions (possible chip/version drift)
multi_pos = (
    merged.groupby("rsid", dropna=False)["position"]
    .nunique(dropna=True)
    .reset_index(name="n_pos")
)
multi_pos_ids = multi_pos[multi_pos["n_pos"] > 1]["rsid"].tolist()

print(f"Files loaded: {[p.name for p in csv_paths]}")
print(f"Total rows: {total_rows:,}")
print(f"Unique (rsid, chr, pos) combos: {unique_triples:,}")
print(f"Duplicate rows detected (same rsid+chr+pos appearing ≥2x): {duplicates.shape[0]:,}")
if multi_pos_ids:
    print(f"⚠️ rsIDs with >1 position across files: {len(multi_pos_ids):,}")
else:
    print("✅ No rsID appears at multiple positions across files.")

# Dedup and sort: chromosome (natural), position, then numeric rs order (rs2 < rs10)
def rs_sort_key(s: str):
    s = "" if s is pd.NA else str(s)
    m = re.fullmatch(r"rs(\d+)", s)
    return (0, int(m.group(1))) if m else (1, s.lower())

unique = (
    merged.drop_duplicates(["rsid", "chromosome", "position"])
    .copy()
    .sort_values(["chromosome", "position", "rsid"], key=lambda col: col.map(lambda x: (x if isinstance(x, str) else x))))


In [ ]:
# Write outputs
# 1) (rsid, chromosome, position) deduped & naturally sorted
def chr_sort_key(c: str):
    c = "" if c is pd.NA else str(c).strip()
    m = re.fullmatch(r"(?:chr)?(\d+)", c, re.I)
    if m:
        return (0, int(m.group(1)))
    c_low = c.lower()
    order = {"x": 23, "y": 24, "mt": 25, "m": 25}
    return (1, order.get(c_low, 99), c_low)

pairs = (
    unique[["rsid", "chromosome", "position"]]
    .dropna(subset=["rsid", "position"])
    .drop_duplicates()
    .copy()
)

# Natural sort: chr (1..22,X,Y,MT), then position, then numeric rs order
pairs["_chr_key"] = pairs["chromosome"].map(chr_sort_key)
pairs["_rs_key"] = pairs["rsid"].map(rs_sort_key)
pairs = pairs.sort_values(by=["_chr_key", "position", "_rs_key"]).drop(columns=["_chr_key", "_rs_key"])

# Save full 3-column TSV (best for dbSNP comparison)
pairs.to_csv("rsid_chr_pos.tsv", sep="\t", index=False)

# Optional: also save 2-column (rsid, position) if you don't need chr
pairs[["rsid", "position"]].to_csv("rsid_pos.tsv", sep="\t", index=False)

print(f"Saved {len(pairs):,} unique (rsid, chr, pos) rows to rsid_chr_pos.tsv")
print("Also wrote rsid_pos.tsv (2 columns).")


In [ ]:
!grep -Eo '^rs[0-9]+' all_rsids_sorted.txt | sed 's/^rs//' | sort -u -n > rsids.numeric

In [ ]:
!LC_ALL=C gsort --parallel=8 -S 4G -t $'\t' -k1,1n ./downloads/b151_SNPChrPosOnRef_105.bcp -o b151.sorted.bcp

In [ ]:
!join -t $'\t' -1 1 -2 1 -o 1.1,2.2,2.3 rsids.numeric b151.sorted.bcp > b151_subset.bcp

In [ ]:
!cut -f1 b151_subset.bcp | sort -u -n > rsids.matched

In [ ]:
!comm -23 rsids.numeric rsids.matched > rsids.unmatched

In [ ]:
!LC_ALL=C sort -u -n rsids.matched -o rsids.matched

In [ ]:
!join -t $'\t' -1 1 -2 1 -o 2.1,2.2,2.3,2.4 rsids.matched b151.sorted.bcp > b151.23andme.bcp

In [ ]:
# compare all the positions in the 23andme files

In [ ]:
from pathlib import Path
import pandas as pd
import io, re

# ---------- config ----------
data_dir = Path(".")
csv_paths = sorted(data_dir.glob("./downloads/family-genome-dataset/*.csv"))
bcp_path  = Path("b151.23andme.bcp")  # tab-separated: snp_id chr_id pos orient

if not csv_paths:
    raise FileNotFoundError(f"No *Genome.csv files under {data_dir.resolve()}")

In [ ]:
# ---------- load dbSNP b151 (sorted bcp) ----------
# Uses header=None to be robust; assign names explicitly.
bcp = pd.read_csv(
    bcp_path,
    sep="\t",
    header=None,
    names=["snp_id", "chr_id", "pos", "orient"],
    dtype=str,
    usecols=[0,1,2,3],
    engine="python",
)

In [ ]:
bcp

In [ ]:
len(bcp)

In [ ]:
# If duplicates exist for the same (snp_id, chr_id), keep the first
bcp = bcp.drop_duplicates(subset=["snp_id", "chr_id"], keep="first").copy()
len(bcp)

In [ ]:
bcp.rename(columns={"pos": "position_bcp"}, inplace=True)

In [ ]:
# ---------- helpers ----------
rs_re = re.compile(r"^rs(\d+)$")

def load_genome_csv(path: Path) -> pd.DataFrame:
    """Load a 23andMe-style CSV, skipping comment/blank lines; return tidy df."""
    with open(path, "r", encoding="utf-8-sig", errors="replace") as f:
        data_lines = [ln for ln in f if not ln.lstrip().startswith("#") and ln.strip()]
    if not data_lines:
        raise ValueError(f"No data rows found in {path}")
    df = pd.read_csv(
        io.StringIO("".join(data_lines)),
        header=None,
        names=["rsid","chromosome","position","genotype"],
        dtype={"rsid": "string", "chromosome": "string", "genotype": "string"},
        low_memory=False,
    )
    # clean
    df["rsid"] = df["rsid"].str.strip()
    df["chromosome"] = df["chromosome"].str.strip()
    df["genotype"] = df["genotype"].str.strip()
    # force integer-like; keep as pandas nullable Int64
    df["position_23"] = pd.to_numeric(df["position"], errors="coerce").astype("Int64")
    
    df.drop(columns=["position"], inplace=True)
    # keep only rs IDs; extract numeric part for joining to bcp
    df["rs_numeric"] = df["rsid"].str.extract(r"^rs(\d+)$", expand=False)
    df = df[~df["rs_numeric"].isna()].copy()
    df["snp_id"] = df["rs_numeric"].astype(str)          # match bcp snp_id (string)
    df.rename(columns={"chromosome": "chr_id"}, inplace=True)
    return df[["rsid","snp_id","chr_id","position_23","genotype"]]

In [ ]:
# ---------- process all genome files ----------
matched_frames = []
missing_frames = []

for p in csv_paths:
    tw = load_genome_csv(p)
    tw["file"] = p.name

    # left join on (snp_id, chr_id) to fetch dbSNP position
    m = tw.merge(bcp[["snp_id","chr_id","position_bcp"]], on=["snp_id","chr_id"], how="left", validate="m:1")

    # split matched vs missing
    matched = m[m["position_bcp"].notna()].copy()
    missing = m[m["position_bcp"].isna()].copy()

    # compare positions for matched
    # ensure numeric compare (coercing bcp string to Int64)
    matched["position_bcp"] = pd.to_numeric(matched["position_bcp"], errors="coerce").astype("Int64")
    matched["positions_agree"] = matched["position_23"].eq(matched["position_bcp"])

    matched_frames.append(matched)
    missing_frames.append(missing)

In [ ]:
# concat all files’ results
matched_all = pd.concat(matched_frames, ignore_index=True) if matched_frames else pd.DataFrame()
missing_all = pd.concat(missing_frames, ignore_index=True) if missing_frames else pd.DataFrame()

In [ ]:
# ---------- save outputs ----------
# Full matched table: rsid, chr, both positions, agree flag, genotype, file
matched_all.loc[:, ["rsid","chr_id","position_23","position_bcp","positions_agree","genotype","file"]] \
    .to_csv("compare_matched.csv", index=False)

# Missing table: rs present in 23andMe but not in dbSNP b151 (for that chr)
missing_all.loc[:, ["rsid","chr_id","position_23","genotype","file"]] \
    .to_csv("compare_missing.csv", index=False)

In [ ]:
print(f"Matched rows: {len(matched_all):,}")
print(f"Missing rows: {len(missing_all):,}")

In [ ]:
# ---------- print all mismatches ----------
mismatches = matched_all[matched_all["positions_agree"] == False]  # noqa: E712
print(f"Mismatches (different positions): {len(mismatches):,}")

i = 0
limit = 10
for _, row in mismatches.loc[:, ["rsid","chr_id","position_23","position_bcp","file"]].iterrows():
    print(f"{row['rsid']}\tchr{row['chr_id']}\t23andme_position={row['position_23']}\tbcp_position={row['position_bcp']}\t({row['file']})")
    i = i + 1
    if i > limit:
        break

In [ ]:
# could be 1 off but when we adjust there are others that don't match

In [ ]:
!for f in ./downloads/**/*.csv; do echo "$f"; cat "$f" | grep 45411941; done

In [ ]:
!awk -F'\t' '$2 == 45411941' ./downloads/clinvar.vcf